<a href="https://colab.research.google.com/github/soberbichler/DHd_Workshop_2026/blob/main/Dolma_Dataset_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset

dataset = load_dataset("allenai/dolma3_mix-5.5T-1125", split="train", streaming=True)


In [ ]:
from datasets import load_dataset

dataset = load_dataset("allenai/dolma3_pool", split="train", streaming=True)

doc = next(iter(dataset))
metadata = doc['metadata']

print("="*70)
for key, value in metadata.items():
    print(f"{key:10s}: {str(value)[:100]}")

In [ ]:
import json

dataset = load_dataset("allenai/dolma3_pool", split="train", streaming=True)

# Get first document
doc = next(iter(dataset))

print("=" * 80)
print("DOCUMENT EXAMPLE")
print("=" * 80)

# Show labels
print("\nLABELS (weborganizer metadata):")
print("-" * 80)
metadata = doc.get('metadata', {})
weborganizer = metadata.get('weborganizer', {})
print(json.dumps(weborganizer, indent=2, ensure_ascii=False))

# Show content
print("\nCONTENT:")
print("-" * 80)
content = doc.get('text', '')
# Show first 1000 characters
print(content[:1000])
if len(content) > 1000:
    print(f"\n... (total length: {len(content)} characters)")

print("\n" + "=" * 80)

In [ ]:
from datasets import load_dataset
from urllib.parse import urlparse
from collections import Counter

dataset = load_dataset("allenai/dolma3_pool", split="train", streaming=True)

MIN_CONFIDENCE = 0.5  # 50% or higher

print(f"Looking for documents with history_and_geography confidence > {MIN_CONFIDENCE}...\n")

history_docs = []
history_domains = Counter()

for i, doc in enumerate(dataset.take(500000)):
    metadata = doc.get('metadata', {})
    weborganizer = metadata.get('weborganizer', {})

    # Safety check: make sure weborganizer is a dict
    if not isinstance(weborganizer, dict):
        continue

    # Get history confidence score (default to 0 if not found)
    history_score = weborganizer.get('__label__history_and_geography', None)

    # Skip if None or not a number
    if history_score is None or not isinstance(history_score, (int, float)):
        continue

    if history_score > MIN_CONFIDENCE:
        text = doc.get('text', '')
        url = metadata.get('warc_url', '')
        top_label = metadata.get('weborganizer_max', '').replace('__label__', '')

        history_docs.append({
            'index': i,
            'score': history_score,
            'top_label': top_label,
            'url': url,
            'text': text[:400],
            'length': len(text)
        })

        # Track domains
        if url:
            domain = urlparse(url).netloc.replace('www.', '')
            history_domains[domain] += 1

print(f"\n\n{'='*70}")
print(f"HIGH-CONFIDENCE HISTORY DOCUMENTS (confidence > {MIN_CONFIDENCE})")
print('='*70)
print(f"Found: {len(history_docs):,} documents ({len(history_docs)/500000*100:.2f}%)")

print(f"\n{'='*70}")
print("TOP DOMAINS")
print('='*70)
for domain, count in history_domains.most_common(15):
    print(f"{domain:50s}: {count:3d}")

print(f"\n{'='*70}")
print("WHAT WERE THESE LABELED AS? (top_label)")
print('='*70)
top_labels = Counter([d['top_label'] for d in history_docs])
for label, count in top_labels.most_common():
    pct = count / len(history_docs) * 100 if history_docs else 0
    print(f"{label:40s}: {count:3d} ({pct:.1f}%)")

print(f"\n{'='*70}")
print("EXAMPLES")
print('='*70)
for i, doc in enumerate(history_docs[:10], 1):
    print(f"\n{i}. Confidence: {doc['score']:.3f} | Labeled as: {doc['top_label']}")
    print(f"   URL: {doc['url'][:70]}")
    print(f"   Length: {doc['length']:,} chars")
    print(f"   Text: {doc['text'][:250]}...")
    print("-" * 70)